In [1]:
import polars as pl
import numpy as np
import pandas as pd
import altair as alt
import os
import wget
import zipfile

from os.path import exists
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

from mapie.classification import MapieClassifier
from mapie.metrics import classification_coverage_score
from mapie.metrics import classification_coverage_score_v2
from mapie.metrics import classification_mean_width_score

In [2]:
beans = "./DryBeanDataset/Dry_Bean_Dataset.xlsx"
base = "https://archive.ics.uci.edu/ml/machine-learning-databases/"
dataset_number = "00602"

if not exists(beans):
    filename = "DryBeanDataset.zip"
    url = f"{base}{dataset_number}/{filename}"
    wget.download(url)
    with zipfile.ZipFile(filename, "r") as zip_ref:
        zip_ref.extractall('./')
    os.remove(filename)

bean_df = pl.read_excel("./DryBeanDataset/Dry_Bean_Dataset.xlsx")
bean_df = bean_df.to_pandas()
le = LabelEncoder()
names = bean_df["Class"].unique()
bean_df["Class"] = le.fit_transform(bean_df["Class"])

y = pd.Series(bean_df["Class"])
X = pd.DataFrame(bean_df.drop("Class", axis=1))
X_train, X_rest1, y_train, y_rest1 = train_test_split(X, y, train_size=10000, random_state=2)
X_test, X_rest2, y_test, y_rest2 = train_test_split(X_rest1, y_rest1, train_size=1000, random_state=42)
X_calib, X_new, y_calib, y_new = train_test_split(X_rest2, y_rest2, train_size=1000, random_state=42)

In [3]:
model = GaussianNB().fit(X_train, y_train)
y_pred = model.predict(X_test)

In [4]:
# For RAPS we first need to uncover the coverage per class and not the avergae classes_data, RAPS is basically like APS but with a regularization term

In [5]:
type(y_new)

pandas.core.series.Series

In [6]:
# the goal is to get coverage close to the 95 % threshold not getting a value as high as possible, while keeping avg set size small
# to artificially lower the coverage of BOMBAY ( as this class was predicted to 100 %) however the score method and the marginal coverage are not adaptive
# adaptive prediction sets I reach differently

In [7]:
# Adaptivity:
# A conformal prediction algorithm is adaptive if it not only achieves marginal coverage but also (apporx.) conditional coverage

In [8]:
# APS
# In this approach we calculate standard non-conformity scores like for score to get the threshold for the calibration set
# then we calculate all class probabilities and to decide which class makes it into the final prediction set (per data point) , we add up the probabilities of a class, e.g.

#cat
#0.5 probability the image is a cat
#lion
#0.3
#dog
#0.1

#if threshold from calibration set is 95 %
#all three classes would be in the prediction set for this specific prediction data point.

# normallly about the las class it is a question to take it in or leave it, as you normally would come above the threshold or stay below

#e.g.
#cat
#0.5 probability the image is a cat
#lion
#0.3
#dog
#0.1
#hamster
#0.06

#the latter would bring us above the threshold, and it is always a question, if we want this or not. Calculation examples indicate that an inclusion 
# leads at least to better results than score bringing most classes above the the treshold or close, but the set sizes are usually larger as wwith standard score , score has the smallest sets on average, but the weakest coverage guarantee
# leaving the last one out, usually leads to worse results than score and should not be done, random is to be preferred, as you can guarantee to have a sample size correction in the coverage added see page 52


In [9]:
# For RAps we would include now a penalty for how many classes we are away from the true label, the penalty is added to the class probability

# classes with low probability are less likely to be included, because the threshold from the score method on the calibration set is reached sooner. But you need some percentage of your calibration data to reserv for the regularization

In [10]:
def class_wise_performance(y_new, y_set, classes):
    df = pd.DataFrame()
    for i in range(len(classes)):
        ynew = y_new.values[y_new.values == i]
        yscore = y_set[y_new.values == i]
        cov = classification_coverage_score(ynew, yscore)
        size = classification_mean_width_score(yscore)
        tmp_df = pd.DataFrame({
            "class": [classes[i]],
            "coverage": [cov],
            "avg set size": [size]
        }, index = [i]
        )
        df = pd.concat([df, tmp_df])
    return df

In [11]:
mapie_score = MapieClassifier(model, cv="prefit", method="raps")

In [12]:
mapie_score.fit(X_calib, y_calib, size_raps=0.2)

MapieClassifier(cv='prefit', estimator=GaussianNB(), method='raps')

In [13]:
y_pred, y_set = mapie_score.predict(X_new, alpha=0.05, include_last_label="randomized")
y_set = np.squeeze(y_set)
cov = classification_coverage_score(y_new, y_set)
setsize = classification_mean_width_score(y_set)
print(cov)
print(setsize)
print(class_wise_performance(y_new, y_set, le.classes_))

0.957169459962756
1.8733705772811917
      class  coverage  avg set size
0  BARBUNYA  0.925287      2.143678
1    BOMBAY  1.000000      2.000000
2      CALI  0.971591      2.142045
3  DERMASON  0.970443      1.453202
4     HOROZ  0.922131      1.872951
5     SEKER  0.961702      1.982979
6      SIRA  0.964630      2.009646
